In [7]:
import os
import json
import numpy as np
import pandas as pd
from utils.fillPoseEstimation import customImputer, column_names

In [8]:
def read_json_files(folder_path):
    json_data = []
    
    for filename in os.listdir(folder_path):
        if filename.endswith(".json"):
            file_path = os.path.join(folder_path, filename)
            with open(file_path, "r") as file:
                for line in file:
                    try:
                        line_data = json.loads(line)
                        json_data.append(line_data)
                    except json.JSONDecodeError:
                        print(f"Error parsing JSON line: {line}")
    
    return json_data


In [9]:
file_path = "Videos"
json_data = read_json_files(file_path)

file_path="Videos/Bill Gates The next outbreak Were not ready TED.mp4.openpifpaf.json"

with open(file_path, 'r') as file:
    for line in file:
        try:
            line_data = json.loads(line)
            json_data.append(line_data)
        except json.JSONDecodeError:
            print(f"Error parsing JSON line: {line}")

In [10]:
full_body = []
half_body = []

In [12]:
for item in json_data:
    predictions = item["predictions"]
    for prediction in predictions:
        keypoints = prediction["keypoints"]
        bbox = prediction["bbox"]
        score = prediction["score"]
        category_id = prediction["category_id"]

        keypoints_arr = np.array(keypoints).reshape(-1, 3)
        c_values = keypoints_arr[:, 2]

        if np.any(c_values <= 0):
            keypoints_without_c = keypoints_arr[:, :2]
            if np.sum(c_values <= 0) <= 4:
                imputed_keypoints = customImputer(keypoints_without_c)
                full_body.append(imputed_keypoints.tolist())
            else:
                half_body.append(keypoints_without_c.tolist())
        else:
            keypoints_without_c = keypoints_arr[:, :2]
            full_body.append(keypoints_without_c.tolist())    

In [13]:
half_body = np.array(half_body)
full_body = np.array(full_body)

In [14]:
reshape_half_body = half_body.reshape(-1,34)
reshape_full_body = full_body.reshape(-1,34)

In [15]:
df_full_body = pd.DataFrame(reshape_full_body, columns=column_names)
df_half_body = pd.DataFrame(reshape_half_body, columns=column_names)

In [16]:
df_half_body

,nose_x,nose_y,left_eye_x,left_eye_y,right_eye_x,right_eye_y,left_ear_x,left_ear_y,right_ear_x,right_ear_y,...,right_hip_x,right_hip_y,left_knee_x,left_knee_y,right_knee_x,right_knee_y,left_ankle_x,left_ankle_y,right_ankle_x,right_ankle_y
0,828.04,154.90,836.28,138.73,0.00,0.00,873.73,132.16,0.00,0.00,...,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,823.20,152.88,830.07,135.78,0.00,0.00,865.68,130.07,0.00,0.00,...,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,818.41,152.27,824.43,133.69,0.00,0.00,861.40,127.13,0.00,0.00,...,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,812.72,150.57,819.47,131.78,0.00,0.00,858.62,124.80,0.00,0.00,...,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,805.06,150.37,812.18,130.86,0.00,0.00,851.60,125.38,0.00,0.00,...,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
468983,174.96,232.17,176.34,229.72,172.92,230.08,180.00,230.50,170.95,231.55,...,172.71,273.07,0.0,-4.0,0.0,-4.0,0.0,-4.0,0.0,-4.0
468984,227.10,257.88,228.04,255.09,224.66,255.99,0.00,-4.00,221.04,257.27,...,0.00,-4.00,0.0,-4.0,0.0,-4.0,0.0,-4.0,0.0,-4.0
468985,270.16,253.67,272.09,251.12,268.03,251.29,276.38,251.94,264.76,252.77,...,266.84,304.41,0.0,-4.0,0.0,-4.0,0.0,-4.0,0.0,-4.0
468986,0.00,-4.00,0.00,-4.00,0.00,-4.00,0.00,-4.00,336.29,241.88,...,333.16,284.75,0.0,-4.0,0.0,-4.0,0.0,-4.0,0.0,-4.0


In [17]:
df_full_body

,nose_x,nose_y,left_eye_x,left_eye_y,right_eye_x,right_eye_y,left_ear_x,left_ear_y,right_ear_x,right_ear_y,...,right_hip_x,right_hip_y,left_knee_x,left_knee_y,right_knee_x,right_knee_y,left_ankle_x,left_ankle_y,right_ankle_x,right_ankle_y
0,760.04,147.16,766.50,131.97,610.925427,110.666324,806.830000,125.620000,494.434654,156.476101,...,834.05,411.38,870.090000,571.670000,751.94,593.48,471.989194,325.859188,745.735831,538.004729
1,754.39,143.97,760.70,127.58,751.420000,131.150000,801.280000,121.880000,777.963189,130.670630,...,821.20,397.39,835.220000,575.930000,767.07,564.40,831.103196,367.146843,725.670152,418.947621
2,804.96,129.87,813.34,114.82,802.740000,116.460000,854.670000,110.330000,833.389181,116.796061,...,906.11,407.92,814.860000,529.730000,889.21,559.46,792.885332,260.359239,816.816193,326.357673
3,808.82,129.36,816.95,115.47,642.775900,97.198499,857.620000,111.080000,508.301555,155.944983,...,907.70,402.76,819.180000,526.640000,886.18,552.75,351.415104,236.045007,854.390457,464.622361
4,811.09,128.60,819.17,114.27,644.382113,95.812372,859.900000,111.570000,508.368126,156.587430,...,899.29,400.77,809.690000,548.020000,888.41,567.71,333.541877,244.163920,868.713600,491.284596
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140871,264.26,251.87,265.97,249.19,261.930000,249.540000,269.770000,249.170000,258.290000,249.650000,...,260.60,299.52,282.508345,282.142674,261.55,330.24,313.996706,306.542632,293.186311,313.157112
140872,332.91,238.93,334.42,236.78,330.780000,236.940000,345.691166,237.808168,326.420000,237.790000,...,326.29,282.01,336.050000,307.750000,325.88,308.91,333.070000,327.300000,323.970000,329.980000
140873,334.58,239.43,335.88,237.21,332.270000,237.360000,347.018712,238.083977,328.030000,238.130000,...,327.28,283.02,336.120000,309.790000,326.18,309.83,332.720000,329.950000,323.830000,332.950000
140874,334.87,240.07,336.13,237.77,332.610000,238.090000,338.740000,237.740000,328.640000,238.660000,...,328.48,286.38,338.210000,308.990000,327.91,311.20,358.025685,326.195286,326.250000,332.110000


In [ ]:
df_full_body.to_csv('data/imputed_full_body.csv', index=False)
df_half_body.to_csv('data/imputed_half_body.csv', index=False)